In [12]:
import numpy as np
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_json('/Users/laurenfinkelstein/Metis/Projects/Project 3/credit_payment_default/results-20190430-210539.json')

# Data cleaning

In [3]:
df.head()

,age,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,default_payment_next_month,education_level,id,limit_balance,marital_status,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,predicted_default_payment_next_month,sex
0,39,47174,47974,48630,50803,30789,15874,0,1,242,50000,2,0,0,0,0,0,0,1800,2000,3000,2000,2000,2000,"[{'tables': {'score': '0.85050475597381592', '...",1
1,29,48088,45980,44231,32489,26354,20221,0,1,1822,110000,2,0,0,0,0,0,0,2000,2010,3000,3000,3000,1000,"[{'tables': {'score': '0.93080854415893555', '...",2
2,36,78630,68921,46512,40335,37165,22156,0,1,5046,270000,2,0,0,0,2,0,0,10076,4018,14,2051,2000,0,"[{'tables': {'score': '0.87450653314590454', '...",1
3,45,58180,59134,61156,62377,63832,65099,0,1,7227,130000,1,0,0,0,0,0,0,2886,2908,2129,2354,2366,2291,"[{'tables': {'score': '0.90300285816192627', '...",1
4,24,42058,35340,22110,19837,19855,20151,0,1,9183,50000,2,0,0,0,0,0,0,1367,1606,692,709,721,692,"[{'tables': {'score': '0.856909453868866', 'va...",1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2965 entries, 0 to 2964
Data columns (total 26 columns):
age                                     2965 non-null int64
bill_amt_1                              2965 non-null int64
bill_amt_2                              2965 non-null int64
bill_amt_3                              2965 non-null int64
bill_amt_4                              2965 non-null int64
bill_amt_5                              2965 non-null int64
bill_amt_6                              2965 non-null int64
default_payment_next_month              2965 non-null int64
education_level                         2965 non-null int64
id                                      2965 non-null int64
limit_balance                           2965 non-null int64
marital_status                          2965 non-null int64
pay_0                                   2965 non-null int64
pay_2                                   2965 non-null int64
pay_3                                   2965 non-null

In [6]:
# is there only one entry per unique user id?
# answer = no duplicates

new = df.drop_duplicates()
new.count()
# df_filtered.groupby('id').count()
# df_filtered['id'].value_counts() > 1

TypeError: unhashable type: 'list'

In [7]:
# remove unecessary columns and reorder necessary columns

new_columns = ['default_payment_next_month', 'age', 'sex', 'marital_status', 'education_level',  
               'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6', 
               'pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6', 
               'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6', 
               'limit_balance']

model_data = df[new_columns]

In [8]:
model_data.head()

,default_payment_next_month,age,sex,marital_status,education_level,bill_amt_1,bill_amt_2,bill_amt_3,bill_amt_4,bill_amt_5,bill_amt_6,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,pay_amt_1,pay_amt_2,pay_amt_3,pay_amt_4,pay_amt_5,pay_amt_6,limit_balance
0,0,39,1,2,1,47174,47974,48630,50803,30789,15874,0,0,0,0,0,0,1800,2000,3000,2000,2000,2000,50000
1,0,29,2,2,1,48088,45980,44231,32489,26354,20221,0,0,0,0,0,0,2000,2010,3000,3000,3000,1000,110000
2,0,36,1,2,1,78630,68921,46512,40335,37165,22156,0,0,0,2,0,0,10076,4018,14,2051,2000,0,270000
3,0,45,1,1,1,58180,59134,61156,62377,63832,65099,0,0,0,0,0,0,2886,2908,2129,2354,2366,2291,130000
4,0,24,1,2,1,42058,35340,22110,19837,19855,20151,0,0,0,0,0,0,1367,1606,692,709,721,692,50000


In [10]:
# figuring out what the different variables mean
# pay_0 etc. have additional labels other than labels described in GCP (e.g., 0, -1), but leave as is?

temp_columns = ['default_payment_next_month', 'age', 'sex', 'marital_status', 'education_level',  
               'bill_amt_1',
               'pay_0', 
               'pay_amt_1',
               'limit_balance']

df_temp = model_data[temp_columns]
df_temp.head()
# df_temp.pay_0.value_counts()

,default_payment_next_month,age,sex,marital_status,education_level,bill_amt_1,pay_0,pay_amt_1,limit_balance
0,0,39,1,2,1,47174,0,1800,50000
1,0,29,2,2,1,48088,0,2000,110000
2,0,36,1,2,1,78630,0,10076,270000
3,0,45,1,1,1,58180,0,2886,130000
4,0,24,1,2,1,42058,0,1367,50000


In [11]:
# where pay_columns have a value < 0, change to 0
# ASSUMPTION: if pay columns have a negative value, customer paid on time

pay_columns = ['pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6']

for column in pay_columns:
    model_data.loc[model_data[column] < 0, column] = 0

/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/laurenfinkelstein/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [13]:
with open ('cleaned_cc_default_data', 'wb') as f:
    pickle.dump(model_data, f)